In [6]:
import os

from Utils import FileUtils as filu
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 40)
pd.options.mode.chained_assignment = None
from bs4 import BeautifulSoup
import requests
import re
from requests_html import HTMLSession

In [ ]:
s = HTMLSession()

In [ ]:
def getdata(url):
    headers = {"User-Agent":
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
              "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}
    r = s.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [ ]:
url = "https://whoicf2.whoi.edu/science/B/whalesounds/index.cfm"
base_url = "https://whoicf2.whoi.edu"

soup = getdata(url)

In [ ]:
options = soup.find("form", {"name": "jump1"}).find_all("option")

In [ ]:
pages = list({option.text.replace("\n", "").replace("\t", "")[1:]:
                  option["value"] for option in options}.items())[1:]

In [ ]:
audio_links = {}
for (whale_name, page_link) in pages:
    audio_links[whale_name] = []
    page = getdata(page_link)
    for a in page.find("table").find_all("a", href=True):
        link = a["href"]
        if link.split(".")[-1] == "wav" or link.split(".")[-1] == "mp3":
            path = base_url + link
            audio_links[whale_name].append(path)

In [ ]:
for (name, links) in list(audio_links.items()):
    dir_path = os.path.join(r"C:\datasets\whoi", name)
    print(f"Saving to {dir_path}")
    os.mkdir(dir_path)
    for i, link in enumerate(links):
        format = link.split(".")[-1]
        audio_path = os.path.join(dir_path, name + " " + str(i) + "." + format)
        filu.download_and_save_file(audio_path, link)